In [39]:
import io
import re
import string
import tqdm
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [46]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")
# os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/Labels")
# os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/manual_dataset")
# with open("DATASET.txt") as dataset_obj:
#     codeLines = dataset_obj.read()

In [41]:
!arch #== 'x86_64'

x86_64


#Filtering

In [47]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
            
    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    for line_number in range(len(raw_codeLines)):
        placeHolder = raw_codeLines[line_number]
        placeHolder = space_out(placeHolder, ";")
        placeHolder = space_out(placeHolder, "(")
        placeHolder = space_out(placeHolder, ")")
        placeHolder = space_out(placeHolder, ",")
        if placeHolder.endswith(";"):
            raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        elif not(placeHolder.endswith(";")):
            raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [43]:
encoding_ex = code_preprocessing("guillermo_branch_simple_insecure.c")
codelines = encoding_ex.copy()

In [48]:
codelines

['<start> #include <stdio.h> <end>',
 '<start>  <end>',
 '<start> int main ( void )  <end>',
 '<start> { <end>',
 '<start>     int result = 1 ;  <end>',
 '<start>     if  ( result == 1 )  <end>',
 '<start>     { <end>',
 '<start>         printf ( "Executing critical code...\\n" )  ;  <end>',
 '<start>     } <end>',
 '<start>     else <end>',
 '<start>     { <end>',
 '<start>         printf ( "Exiting out...\\n" )  ;  <end>',
 '<start>         return 1 ;  <end>',
 '<start>     } <end>',
 '<start>  <end>',
 '<start>     return 0 ;  <end>',
 '<start> } <end>']

In [49]:
processed_codeLines = code_preprocessing("DATASET.txt")
# processed_secure_codeLines = code_preprocessing("SECURE_DATA.txt")
# processed_insecure_codeLines = code_preprocessing("INSECURE_DATA.txt")


In [50]:
len(processed_codeLines)

5306

In [ ]:
processed_secure_codeLines[2]

'<start> #include <Compiler.h> <end>'

In [25]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token

for line in encoding_ex:
  tokens = list(line.split())
  for token in tokens:
    if token not in vocab:
      vocab[token] = index
      index += 1
# vocab_size = len(vocab)
# inverse_vocab = {index: token for token, index in vocab.items()}
# print(inverse_vocab)
len(vocab.keys())

27

In [26]:
vocab

{'<pad>': 0,
 '<start>': 1,
 '#include': 2,
 '<stdio.h>': 3,
 '<end>': 4,
 'int': 5,
 'main': 6,
 '(': 7,
 'void': 8,
 ')': 9,
 '{': 10,
 'result': 11,
 '=': 12,
 '1': 13,
 ';': 14,
 'if': 15,
 '==': 16,
 'printf': 17,
 '"Executing': 18,
 'critical': 19,
 'code...\\n"': 20,
 '}': 21,
 'else': 22,
 '"Exiting': 23,
 'out...\\n"': 24,
 'return': 25,
 '0': 26}

In [27]:
def one_hot_encode(categories, category_to_index):
    gloabl_OHV_dictionary = {}
    num_categories = len(categories)
    num_indices = len(category_to_index)
    one_hot_vectors = np.zeros((num_categories, num_indices))
    for i, category in enumerate(categories):
        index = category_to_index.get(category)
        if index is not None:
            one_hot_vectors[i, index] = 1
            gloabl_OHV_dictionary[category] = one_hot_vectors[i]
    return one_hot_vectors, gloabl_OHV_dictionary

global_one_hot_vectors, gloabl_OHV_dictionary = one_hot_encode(vocab.keys(), vocab)
# print(global_one_hot_vectors)
# global_one_hot_vectors[0].shape
# gloabl_OHV_dictionary['<Compiler.h>']


Secure Reconstruction

In [ ]:
for line in range(len(processed_secure_codeLines)):
    processed_secure_codeLines[line] = processed_secure_codeLines[line].split()
    for element_index in range(len(processed_secure_codeLines[line])):
        processed_secure_codeLines[line][element_index] = gloabl_OHV_dictionary[processed_secure_codeLines[line][element_index]]

Insecure Reconstruction

In [ ]:
for line in range(len(processed_insecure_codeLines)):
    processed_insecure_codeLines[line] = processed_insecure_codeLines[line].split()
    for element_index in range(len(processed_insecure_codeLines[line])):
        processed_insecure_codeLines[line][element_index] = gloabl_OHV_dictionary[processed_insecure_codeLines[line][element_index]]

In [28]:
for line in range(len(encoding_ex)):
    encoding_ex[line] = encoding_ex[line].split()
    for element_index in range(len(encoding_ex[line])):
        encoding_ex[line][element_index] = gloabl_OHV_dictionary[encoding_ex[line][element_index]]

Padding Implementation

In [29]:
# max_len = 0
# for i in (processed_secure_codeLines):
#     if  len(i) > max_len:
#         max_len = (len(i))
# for i in enumerate(processed_secure_codeLines):
#     # print(i[1])
#     for iter in range(0, max_len - len(i[1])):
        
#         processed_secure_codeLines[i[0]].insert(-1, gloabl_OHV_dictionary['<pad>'])

# max_len = 0
# for i in (processed_insecure_codeLines):
#     if  len(i) > max_len:
#         max_len = (len(i))
# for i in enumerate(processed_insecure_codeLines):
#     for iter in range(0, max_len - len(i[1])):
#         processed_insecure_codeLines[i[0]].insert(-1, gloabl_OHV_dictionary['<pad>'])

max_len = 0
for i in (encoding_ex):
    if  len(i) > max_len:
        max_len = (len(i))
for i in enumerate(encoding_ex):
    for iter in range(0, max_len - len(i[1])):
        encoding_ex[i[0]].insert(-1, gloabl_OHV_dictionary['<pad>'])

In [38]:
for i in enumerate(encoding_ex):
    print(codelines[i[0]])
    for j in i[1]:
        print(j)
    

<start> #include <stdio.h> <end>
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
<start>  <end>
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 

In [34]:
for j in i[1][0]:
    print(j)

0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
